<h1>Segmenting and Clustering Neighborhoods in Toronto (Part 1&2)

<h3>For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

<h3>1. import libraries

In [22]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h3>2. Scrap data from Wikipedia page into a DataFrame

In [23]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# send a GET request and request object to html of website

In [25]:
# parse data from html to beautifulsoup object
soup = BeautifulSoup(source,'html.parser')

In [27]:
# create empty list to store data
postalCode = []
borough = []
neighborhood = []

In [29]:
# append data to lists 
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if (len(cells)>0) :
        postalCode.append(cells[0].text.rstrip('\n'))
        borough.append(cells[1].text.rstrip('\n'))
        neighborhood.append(cells[2].text.rstrip('\n'))
        

In [30]:
# create new df
trt_df = pd.DataFrame({"PostalCode": postalCode,
                       "Borough": borough,
                       "Neighborhood": neighborhood})

trt_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3>3. Drop cells of "Not Assigned" in Borough

In [32]:
trt_df['Borough'].replace('Not assigned',np.nan, inplace = True)
trt_df.dropna(subset = ['Borough'],inplace =True)
trt_df.reset_index(drop = True,inplace = True)
trt_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3> 4. Group neighborhoods in the same borough

In [34]:
final_df = trt_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
final_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3> 5. For Neighborhood="Not assigned", make the value the same as Borough

In [36]:
for index,row in final_df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

# final_df.loc[final_df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = final_df['Borough']
final_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3> 6. Check this dataframe

In [50]:
# create a test dataframe
test_df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for pc in test_list:
    test_df = test_df.append(final_df[final_df["PostalCode"]==pc], ignore_index=True)
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


<h3>7. Print the number of rows of the cleaned dataframe 

In [51]:
final_df.shape

(103, 3)

<h3> 8. Load the coordinates from the csv file on Coursera

In [52]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.rename(columns = {"Postal Code": "PostalCode"},inplace = True)
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3> 9. Merge two tables (added coordinates to previous table)

In [56]:
trt_df = final_df.merge(geo_df, on = "PostalCode", how = "left" )
trt_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h3> 10. Check this dataframe

In [57]:
# create a test dataframe
test_df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"])

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for pc in test_list:
    test_df = test_df.append(trt_df[trt_df["PostalCode"]==pc], ignore_index=True)

test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
